In [184]:
import numpy as np
from sklearn.feature_extraction.image import extract_patches_2d
from skimage.util.shape import view_as_blocks
from skimage.util import view_as_windows
from numpy.lib import stride_tricks
import cv2

np.set_printoptions(precision=3)

class PatchMatch(object):
    def __init__(self, a, b, patch_size):
        assert a.shape == b.shape, "Dimensions were unequal for patch-matching input"
        
        self.orig_shape = a.shape
        self.patch_size = patch_size
        self.offset_from_edge = self.patch_size // 2
        offset_from_edge = self.offset_from_edge
        self.a = a
        self.b = b
        
        self.a_padded = np.pad(a,pad_width=((0,0),(offset_from_edge,offset_from_edge),(offset_from_edge,offset_from_edge)),mode='constant')
        
        self.b_padded = np.pad(b,pad_width=((0,0),(offset_from_edge,offset_from_edge),(offset_from_edge,offset_from_edge)),mode='constant')
        
        
        self.nnf = np.ndarray(shape=(2, a.shape[1], a.shape[2]),dtype=np.int32)
        self.nnd = np.ndarray(shape=(a.shape[1], a.shape[2]))

        print(self.a.shape)
        print(self.b.shape)
        print("SHAPE OF PADDED a {}".format(self.a_padded.shape))
        print("SHAPE OF PADDED b {}".format(self.b_padded.shape))


    
        for i in range(0, a.shape[1] ): #loop  through y
            for j in range(0, a.shape[2]): # loop through x
                self.nnf[0][i][j] = int(np.random.randint(0,a.shape[2]))
                self.nnf[1][i][j] = int(np.random.randint(0,a.shape[1]))
                self.nnd[i][j] = self.calculate_distance(j,i,self.nnf[0][i][j],self.nnf[1][i][j])
                
    def clean_coords(self,x,y):
        assert x >= self.offset_from_edge and x < self.orig_shape[2] , "X coordinate off"
        assert y >= self.offset_from_edge and y < self.orig_shape[1] , "Y coordinate off"
        
        
    def calculate_distance(self,ax,ay,bx,by):

        patch_a = self.get_patch_for_coords(self.a_padded,ax,ay)
        patch_b = self.get_patch_for_coords(self.b_padded,bx,by)
#         print(patch_a.shape)
#         print(patch_b.shape)
        assert patch_a.shape == patch_b.shape, " ax {}  ay {}  bx {}  by {} a.shape = {}  b.shape={}".format(ax,ay,bx,by,patch_a.shape,patch_b.shape)


        diff = patch_a - patch_b
        distances = np.linalg.norm(diff,axis=0)
#         print("DISTANCE {}".format(distances.size))
        average = np.sum(distances)/(distances.size)
        return average
        
        
    
    def get_patch_for_coords(self, arr, x, y):
        """
        Get a patch of at max patch_size X patch_size.
        x and y denotes the center of the patch

        :arr: an array of dimensions C * H * W 
        :return:
        """
#         x,y = self.clip_coords(x,y)
        
        return arr[:,y: y + self.patch_size,
                     x : x+self.patch_size ]
        

        return arr[ : , y : y+self.offset_from_edge, x:x+self.offset_from_edge]
    
    def propagate(self):
        old_nnd = self.nnd.copy()
        print(self.nnd)
        print("-"*10)
        for i in range(1,self.orig_shape[1]): # loop through ys
            for j in range(1,self.orig_shape[2]):# loop through xs
                current_pos_distance = self.nnd[i][j] 
                
                horiz_x = j - 1
                horiz_y = i
                horiz_dist = self.calculate_distance(j,i,self.nnf[0][horiz_y][horiz_x],self.nnf[1][horiz_y][horiz_x])
                
                vert_x = j 
                vert_y = i - 1  
                vert_dist = self.calculate_distance(j,i,self.nnf[0][vert_y][vert_x],self.nnf[1][vert_y][vert_x])

                best_dist = min(current_pos_distance,horiz_dist,vert_dist)
                
                if best_dist == current_pos_distance:
                    best_x = j
                    best_y = i
                elif best_dist == horiz_dist:
                    best_x = horiz_x
                    best_y = horiz_y
                elif best_dist == vert_dist:
                    best_x = vert_x
                    best_y = vert_y
                    
                rand_d = min(self.a.shape[1]//2, self.a.shape[2]//2)
                
                while rand_d > 0:
                        xmin = max(best_x - rand_d, 0)
                        xmax = min(best_x + rand_d, self.b.shape[2])
                        ymin = max(best_y - rand_d, 0)
                        ymax = min(best_y + rand_d, self.b.shape[1])

                        rand_x = np.random.randint(xmin, xmax)
                        rand_y = np.random.randint(ymin, ymax)
                        val = self.calculate_distance(j,i, rand_x, rand_y)
                        if val < best_dist:
                            best_x, best_y, best_dist = rand_x, rand_y, val
                        rand_d = rand_d // 2    
              
                self.nnf[0][i][j] = best_x 
                self.nnf[1][i][j] = best_y
                self.nnd[i][j] = best_dist
        print(self.nnd)
        print("_"*10)
        print(old_nnd-self.nnd)
            
    
    def get_loss_value(self):
        return np.sum(self.nnd)
    def clip_coords(self,x,y):
        return max(x,0) , max(y,0)
        
    def back_propagate(self):
            old_nnd = self.nnd.copy()
            print(self.nnd)
            print("-"*10)
            for i in range(self.orig_shape[1]-2,0,-1): # loop through ys
                for j in range(self.orig_shape[2]-2,0,-1):# loop through xs
                    current_pos_distance = self.nnd[i][j] 

                    horiz_x = j + 1
                    horiz_y = i
                    horiz_dist = self.calculate_distance(j,i,self.nnf[0][horiz_y][horiz_x],self.nnf[1][horiz_y][horiz_x])

                    vert_x = j 
                    vert_y = i + 1  
                    vert_dist = self.calculate_distance(j,i,self.nnf[0][vert_y][vert_x],self.nnf[1][vert_y][vert_x])

                    best_dist = min(current_pos_distance,horiz_dist,vert_dist)

                    if best_dist == current_pos_distance:
                        best_x = j
                        best_y = i
                    elif best_dist == horiz_dist:
                        best_x = horiz_x
                        best_y = horiz_y
                    elif best_dist == vert_dist:
                        best_x = vert_x
                        best_y = vert_y

                    rand_d = min(self.a.shape[1]//2, self.a.shape[2]//2)

                    while rand_d > 0:
                            xmin = max(best_x - rand_d, 0)
                            xmax = min(best_x + rand_d, self.b.shape[2])
                            ymin = max(best_y - rand_d, 0)
                            ymax = min(best_y + rand_d, self.b.shape[1])

                            rand_x = np.random.randint(xmin, xmax)
                            rand_y = np.random.randint(ymin, ymax)
                            val = self.calculate_distance(j,i, rand_x, rand_y)
                            if val < best_dist:
                                best_x, best_y, best_dist = rand_x, rand_y, val
                            rand_d = rand_d // 2    

                    self.nnf[0][i][j] = best_x 
                    self.nnf[1][i][j] = best_y
                    self.nnd[i][j] = best_dist
            print(self.nnd)
            print("_"*10)
            print(old_nnd-self.nnd)



In [186]:
# test_a = np.random.randint(low=0,high=10,size=(7,100, 100))
# test_b = np.random.randint(low=0,high=10,size=(7,100, 100))

x = cv2.imread("/Users/harshvardhangupta/Deep-Image-Analogy/notebooks/Unknown.jpeg").transpose(2,1,0)
y = cv2.imread("/Users/harshvardhangupta/Deep-Image-Analogy/notebooks/Unknown.jpeg").transpose(2,1,0)
# x = x[:,:7,:7]
# y = y[:,:7,:7]


print(x.shape)
print("-")
# test_a = test_a.transpose((2,0,1))
# print(test_a.shape)

pm = PatchMatch(x,y,3)
pm.propagate()
pm.back_propagate()
pm.propagate()
# pm.back_propagate()
pm.propagate()
# pm.get_loss_value()
pm.propagate()



(3, 275, 183)
-
(3, 275, 183)
(3, 275, 183)
SHAPE OF PADDED a (3, 277, 185)
SHAPE OF PADDED b (3, 277, 185)
[[ 226.76   176.395  188.743 ...,   85.677  396.375  376.824]
 [ 302.147  252.867  241.609 ...,  197.85   219.149  367.609]
 [ 243.694  245.037  252.998 ...,  143.317  263.557   60.978]
 ..., 
 [ 307.152   93.115  353.186 ...,  285.514  107.466  356.265]
 [  61.661  373.501  330.607 ...,  237.239  359.293  344.766]
 [ 166.751  320.776  358.142 ...,  192.403  381.784   93.362]]
----------
[[  2.268e+02   1.764e+02   1.887e+02 ...,   8.568e+01   3.964e+02
    3.768e+02]
 [  3.021e+02   1.847e+02   1.941e+02 ...,   0.000e+00   0.000e+00
    0.000e+00]
 [  2.437e+02   0.000e+00   1.809e+02 ...,   0.000e+00   0.000e+00
    0.000e+00]
 ..., 
 [  3.072e+02   0.000e+00   3.333e-01 ...,   0.000e+00   0.000e+00
    0.000e+00]
 [  6.166e+01   4.444e-01   4.444e-01 ...,   0.000e+00   0.000e+00
    0.000e+00]
 [  1.668e+02   8.505e+00   8.505e+00 ...,   0.000e+00   0.000e+00
    8.686e+01]]
_

In [174]:
pm.calculate_distance(0,0,123,12)

321.29738051516563

In [ ]:
pm.get_patch_for_coords(x,0,0)

(3, 275, 183)


In [ ]:
x[:,0:3,0:3]

In [ ]:
pm.a[:,7+2,8+2]

In [46]:
pm.nnf[:,0,0] = [2,2]

In [47]:
pm.nnf[:,0,0]

array([2, 2], dtype=int32)

In [97]:
x = cv2.imread("/Users/harshvardhangupta/Deep-Image-Analogy/notebooks/Unknown.jpeg").transpose(2,1,0)
# x = x[:,:4,:4]
# x

array([[[  0, 197,  90,   5],
        [  0, 190,  82,  39],
        [ 63, 175,   0,  89],
        [122, 163,   7, 121]],

       [[ 19, 214,  89,   0],
        [ 33, 207,  81,  30],
        [ 96, 194,   0,  81],
        [156, 182,   6, 113]],

       [[ 28, 223,  99,   6],
        [ 42, 216,  91,  40],
        [105, 202,   7,  88],
        [162, 190,  16, 120]]], dtype=uint8)

In [99]:
np.pad(x,pad_width=((0,0),(1,1),(1,1)),mode='constant')

array([[[  0,   0,   0,   0,   0,   0],
        [  0,   0, 197,  90,   5,   0],
        [  0,   0, 190,  82,  39,   0],
        [  0,  63, 175,   0,  89,   0],
        [  0, 122, 163,   7, 121,   0],
        [  0,   0,   0,   0,   0,   0]],

       [[  0,   0,   0,   0,   0,   0],
        [  0,  19, 214,  89,   0,   0],
        [  0,  33, 207,  81,  30,   0],
        [  0,  96, 194,   0,  81,   0],
        [  0, 156, 182,   6, 113,   0],
        [  0,   0,   0,   0,   0,   0]],

       [[  0,   0,   0,   0,   0,   0],
        [  0,  28, 223,  99,   6,   0],
        [  0,  42, 216,  91,  40,   0],
        [  0, 105, 202,   7,  88,   0],
        [  0, 162, 190,  16, 120,   0],
        [  0,   0,   0,   0,   0,   0]]], dtype=uint8)

In [165]:
pm.get_patch_for_coords(pm.a_padded,5,5)

array([[[ 0, 32],
        [ 4, 15]],

       [[ 3, 35],
        [ 6, 19]],

       [[ 0, 26],
        [ 0,  8]]], dtype=uint8)

In [144]:
x[:,y+self.offset_from_edge: y + self.patch_size , x+self.offset_from_edge : x+self.patch_size]

NameError: name 'self' is not defined

In [126]:
x[:,36+1 : 36 + 4 , 36+1 : 36+4]

array([[[92, 92, 92],
        [94, 94, 94],
        [95, 95, 95]],

       [[54, 54, 54],
        [56, 56, 56],
        [57, 57, 57]],

       [[36, 36, 36],
        [38, 38, 38],
        [39, 39, 39]]], dtype=uint8)

In [127]:
ax 36  ay 0  bx 182  by 8

SyntaxError: invalid syntax (<ipython-input-127-18d286b8d8a4>, line 1)

In [145]:
x[:,8+1:8+3,182+1:182+3]

array([], shape=(3, 2, 0), dtype=uint8)

In [146]:
x.shape

(3, 275, 183)

In [153]:
padded_x = np.pad(x,pad_width=((0,0),(1,1),(1,1)),mode='constant')

In [142]:
x.shape

(3, 275, 183)

In [154]:
padded_x

array([[[  0,   0,   0, ...,   0,   0,   0],
        [  0,   0, 197, ...,  48,  48,   0],
        [  0,   0, 190, ...,  48,  48,   0],
        ..., 
        [  0, 231, 231, ...,  48,  48,   0],
        [  0, 231, 231, ...,  48,  48,   0],
        [  0,   0,   0, ...,   0,   0,   0]],

       [[  0,   0,   0, ...,   0,   0,   0],
        [  0,  19, 214, ...,  22,  22,   0],
        [  0,  33, 207, ...,  22,  22,   0],
        ..., 
        [  0, 251, 251, ...,  22,  22,   0],
        [  0, 251, 251, ...,  22,  22,   0],
        [  0,   0,   0, ...,   0,   0,   0]],

       [[  0,   0,   0, ...,   0,   0,   0],
        [  0,  28, 223, ...,  22,  22,   0],
        [  0,  42, 216, ...,  22,  22,   0],
        ..., 
        [  0, 255, 255, ...,  22,  22,   0],
        [  0, 255, 255, ...,  22,  22,   0],
        [  0,   0,   0, ...,   0,   0,   0]]], dtype=uint8)

In [162]:
padded_x[:,8:8+4,182:182+3]

array([[[48, 48,  0],
        [48, 48,  0],
        [49, 49,  0]],

       [[22, 22,  0],
        [22, 22,  0],
        [23, 23,  0]],

       [[22, 22,  0],
        [22, 22,  0],
        [23, 23,  0]]], dtype=uint8)

In [ ]:
np.pad